In [29]:
import sys
sys.path.append('../')
sys.path.append('./src')
import os
import os.path as osp
import numpy as np
import pyvista as pv
pv.set_jupyter_backend('trame')
pv.start_xvfb()
from time import time
import networkx as nx
from pymeshfix._meshfix import PyTMesh

from vascular_prep import io
from vascular_prep import utils as ut

tic = time()

In [30]:
##------------------------ Options
input_aorta_model_file = "../../data/labels/aorta/0002.nii.gz"
input_lv_model_file = "../../data/labels/left_ventricle/0002.nii.gz"
output_dir = '../../data/outputs/0002'
os.makedirs(output_dir, exist_ok=True)
delta = 0.05 # incremental parameter for iterative thresholding
show_plots = False

In [31]:
##------------------------ Read input file(s)
ao_surf = io.read_file(input_aorta_model_file)

if input_lv_model_file is not None:
    lv_surf = io.read_file(input_lv_model_file)
else:
    raise NotImplementedError()

Reading data... ImageData input...   2.91s
Reading data... ImageData input...   1.75s


In [32]:
##------------------------ Remeshing and smoothing
ao_surf = ut.meshfix(ao_surf)
ao_surf = ut.polydata_smoothing(ao_surf, lamb=0.95, iterations=100)#.smooth(500)
ao_surf = ut.mmg_remesh(ao_surf, hausd=0.5, hmax=1.5, hmin=0.9, max_aspect_ratio=5, max_iter=3, verbose=False)

lv_surf.decimate(0.6)

if show_plots:
    pl = pv.Plotter()
    pl.add_mesh(ao_surf, color='w', opacity=1, show_edges=True)
    pl.add_mesh(lv_surf, color='pink',opacity=0.5)
    pl.show_axes()
    pl.show()

Meshfix...   4.92s
Smoothing...   10.90s


  ** 25683.sol  NOT FOUND. USE DEFAULT METRIC.


In [33]:
##------------------------ Detect aortic valve plane (inlet)
print('Source point detection...')
lv_surf.compute_implicit_distance(ao_surf, inplace=True)
av_region = lv_surf.threshold_percent(percent=0.1, scalars='implicit_distance', invert=True)
av_center = np.array(av_region.center)
av_radius = np.mean([np.linalg.norm(pt - av_center) for pt in av_region.points])
av_normal = pv.fit_plane_to_points(av_region.points).point_normals.mean(0)

# av_normal should point toward the ascendin aorta, not toward the left ventricle
lv_com = lv_surf.center
p1 = av_center + av_normal*20
if np.linalg.norm(p1 - lv_com) < np.linalg.norm(av_center - lv_com):
    av_normal = -1 * av_normal

if show_plots:
    pl = pv.Plotter()
    pl.add_mesh(ao_surf, color='w', opacity=0.5)
    pl.add_points(av_center, color='g', point_size=10, render_points_as_spheres=True)
    pl.add_mesh(pv.Arrow(start=av_center, direction=av_normal, scale=20), color='k')
    pl.show_axes()
    pl.show()

Source point detection...


In [34]:
##------------------------ Detect centerline keypoints
print('Centerline seed detection...')
source_pt, _ = ao_surf.ray_trace(av_center-20*av_normal, av_center+20*av_normal)
st_id = ao_surf.find_closest_point(source_pt.T)
source_pt = ao_surf.points[st_id]

# Detect aortic centerline endpoints
G_ao = ut.convert_triangle_mesh_to_graph(ao_surf)
shortest_paths_lengths = nx.single_source_shortest_path_length(G_ao, st_id)
ao_surf['Geodesics'] = np.zeros((ao_surf.n_points,))
for j in range(ao_surf.n_points):
    geo_path = shortest_paths_lengths[j]
    ao_surf['Geodesics'][j] = geo_path
ao_surf['Geodesics_n'] = (ao_surf['Geodesics'] - np.min(ao_surf['Geodesics'])) / np.ptp(ao_surf['Geodesics'])
desc_ao_pt = ao_surf.points[np.argmax(ao_surf['Geodesics_n'])]
surf_with_geodesics = ao_surf.copy()

# Detect ends: recursive thresholding of geodesic distance to ostia
delta = delta
cand_outlets = []
for delta in np.arange(delta, 1, delta):
    thr = ao_surf.threshold_percent(1-delta, scalars='Geodesics_n').connectivity()
    for j in np.unique(thr['RegionId']):
        thr_connected = thr.threshold([j, j+0.5], scalars='RegionId')
        cand_outlets.append(np.array(thr_connected.points[np.argmax(thr_connected['Geodesics_n'])]))
cand_outlets = np.unique(cand_outlets, axis=0)

# descending aorta point
toremove = np.argmin([np.linalg.norm(desc_ao_pt - c) for c in cand_outlets])
cand_outlets = np.array([cand_outlets[i] for i in range(len(cand_outlets)) if i != toremove])
print(f'A total of {len(cand_outlets) + 1} outlets were detected')

if show_plots:
    pl = pv.Plotter()
    pl.add_mesh(ao_surf, color='w', opacity=0.5)
    pl.add_points(source_pt, color='g', point_size=10, render_points_as_spheres=True)
    pl.add_points(cand_outlets, color='r', point_size=15, render_points_as_spheres=True)
    pl.add_points(desc_ao_pt, color='m', point_size=15, render_points_as_spheres=True)
    pl.show_axes()
    pl.show()

Centerline seed detection...
A total of 5 outlets were detected


In [35]:
##------------------------ Centerlines extraction
print('Computing centerlines...')
parentCenterline = ut.extract_centerline(ao_surf, list(source_pt), list(desc_ao_pt), resampling=0.1, appendEndPoints=0)

# Branches centerline extraction
centerlines = ut.extract_centerline(ao_surf, list(source_pt), [c for pt in cand_outlets for c in pt],
                                        resampling=0.1, appendEndPoints=0).connectivity()

if show_plots:
    pl = pv.Plotter()
    pl.add_mesh(ao_surf, color='w', opacity=0.5)
    pl.add_mesh(parentCenterline, color='m', line_width=5,)
    pl.add_mesh(centerlines, color='r', line_width=5)
    pl.show_axes()
    pl.show()

Computing centerlines...
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...

In [36]:
##------------------------ Proximal extension with extrusion
print('\nClipping aorta proximally...')

# this loop allows to gradually move a disc along the av_normal that intersects the ascending aorta.
n_cells = []
dists_to_av = []
offset_array = np.arange(-20, 20, 0.5)
slices = []
for i in offset_array:
    disc_ = pv.Disc(center=av_center+i*av_normal, inner=0, outer=av_radius*3, normal=av_normal, r_res=20, c_res=20)
    col, n_contacts = ao_surf.collision(disc_)
    n_cells.append(n_contacts)

selected_center = av_center + offset_array[np.min(np.where(np.array(n_cells) > 0)) - 1] * av_normal

offset = 40
cut_cyl = pv.Cylinder(center=selected_center,
                      radius=av_radius,
                      direction=av_normal,
                      height=offset,
                      resolution=200,
                      capping=True).triangulate().subdivide(2)
cut_cyl_inlet = cut_cyl.copy(deep=True)
clipped_ao = ao_surf.clip_surface(cut_cyl, invert=False)
clipped_ao_clean = pv.PolyData(clipped_ao.points, faces=clipped_ao.faces)

edges = clipped_ao_clean.extract_feature_edges(80).connectivity()
edge_dists_to_av = []
for i in np.unique(edges['RegionId']):
    thr = edges.threshold([i, i+0.5], scalars='RegionId')
    if thr.n_points > 10:
        edge_dists_to_av.append(np.linalg.norm(thr.center - np.array(av_center)))
    else:
        edge_dists_to_av.append(1000)
idx = np.unique(edges['RegionId'])[np.argmin(edge_dists_to_av)]
edge_2keep = edges.threshold([idx, idx+0.5], scalars='RegionId').extract_surface()

# extrude edge
extr = edge_2keep.extrude(vector=-av_normal*20, capping=False).triangulate()#.subdivide(2)
combined_surf = clipped_ao.merge(extr).clean()


Clipping aorta proximally...


In [37]:
##------------------------ Clip aorta at inlet
combined_surf.save(f'aorta.stl')
mfix = PyTMesh(False)  # False removes extra verbose output
mfix.load_file(f'aorta.stl')
os.remove(f'aorta.stl')
mfix.fill_small_boundaries(nbe=100, refine=True)
vert, faces = mfix.return_arrays()
triangles = np.empty((faces.shape[0], 4), dtype=faces.dtype)
triangles[:, -3:] = faces
triangles[:, 0] = 3
mesh = pv.PolyData(vert, triangles)
#mesh = ut.mmg_remesh(mesh, hausd=0.5, hmax=1.5, hmin=0.9, max_aspect_ratio=100, max_iter=3, verbose=False).connectivity()

# clip combined surf
cut_cyl2 = pv.Cylinder(center=selected_center-av_normal*offset/2,
                      radius=av_radius*2,
                      direction=av_normal,
                      height=offset*0.95,
                      resolution=200,
                      capping=True).triangulate().subdivide(4)
cut_cyl_inlet = cut_cyl2.copy(deep=True)
combined_surf = mesh.clip_surface(cut_cyl2, invert=False).connectivity()
combined_surf = ut.mmg_remesh(combined_surf, hausd=0.1, hmax=1.5, hmin=0.3, max_aspect_ratio=100, max_iter=3, verbose=False).connectivity()

  ** 25683.sol  NOT FOUND. USE DEFAULT METRIC.


Warning: Meshio doesn't know keyword RequiredVertices. Skipping.

Warning: Meshio doesn't know keyword Ridges. Skipping.

Warning: Meshio doesn't know keyword Tangents. Skipping.

Warning: Meshio doesn't know keyword TangentAtVertices. Skipping.

In [38]:
##------------------------ Clip aorta at branches
ao_surf_clipped_branches = combined_surf.copy(deep=True)
cut_cyl_branches = []
print('Clipping aorta at branches...')
for cid in np.unique(centerlines['RegionId']):
    dummy = pv.PolyData(combined_surf.points, combined_surf.faces)
    cl = centerlines.threshold([cid, cid+0.5], scalars='RegionId').extract_surface()
    cutoff_idx = np.max(np.where(cl['Abscissas'] < np.max(cl['Abscissas'])*0.95))
    slice_ = dummy.slice(normal=cl['FrenetTangent'][cutoff_idx], origin=cl.points[cutoff_idx]).connectivity()
    dists = []
    for j in np.unique(slice_['RegionId']):
        thr = slice_.threshold([j, j+0.5], scalars='RegionId', preference='point').extract_surface()
        dists.append(np.linalg.norm(np.array(thr.center) - cl.points[cutoff_idx]))
    region2keep = np.unique(slice_['RegionId'])[np.argmin(dists)]
    thr = slice_.threshold([region2keep, region2keep+0.5], scalars='RegionId', preference='point').extract_surface()
    radius_for_cut = np.max([np.linalg.norm(pt - cl.points[cutoff_idx]) for pt in thr.points])

    # create cylinder for cut
    cut_cyl = pv.Cylinder(center=cl.points[cutoff_idx] + cl['FrenetTangent'][cutoff_idx] * 2,
                          direction=cl['FrenetTangent'][cutoff_idx],
                          radius=radius_for_cut*1.2,
                          height=5)
    cut_cyl_branches.append(cut_cyl)

    # clip surface with plane
    ao_surf_clipped_branches = ao_surf_clipped_branches.clip_surface(cut_cyl, invert=False).connectivity('largest')

Clipping aorta at branches...


In [39]:
##------------------------ Clip descending aorta
if parentCenterline['Abscissas'][0] < parentCenterline['Abscissas'][2]:
    cutoff_idx = np.max(np.where(parentCenterline['Abscissas'] < np.max(parentCenterline['Abscissas'])*0.9))
else:
    cutoff_idx = np.min(np.where(parentCenterline['Abscissas'] < np.max(parentCenterline['Abscissas']) * 0.9))

# create cylinder for cut
cut_cyl = pv.Cylinder(center=parentCenterline.points[cutoff_idx] + parentCenterline['FrenetTangent'][cutoff_idx] * 2,
                      direction=parentCenterline['FrenetTangent'][cutoff_idx],
                      radius=2.2 * parentCenterline['MaximumInscribedSphereRadius'][cutoff_idx],
                      height=3)
cut_cyl_desc = cut_cyl.copy(deep=True)

# clip surface with plane
ao_surf_clipped_all = ao_surf_clipped_branches.clip_surface(cut_cyl, invert=False).connectivity('largest')
ao_surf_clipped_all.save(f'aorta.stl')

mfix = PyTMesh(False)  # False removes extra verbose output
mfix.load_file(f'aorta.stl')
os.remove(f'aorta.stl')
mfix.fill_small_boundaries(nbe=20, refine=True)
vert, faces = mfix.return_arrays()
triangles = np.empty((faces.shape[0], 4), dtype=faces.dtype)
triangles[:, -3:] = faces
triangles[:, 0] = 3

mesh = pv.PolyData(vert, triangles)

In [40]:
##------------------------ Assign tags to subregions
print('Creating subregion tags...')
ao_tagged = mesh.copy(deep=True)
ao_tagged['tags'] = np.zeros(mesh.n_points)

edges = ao_tagged.extract_feature_edges(80, boundary_edges=True, feature_edges=False, manifold_edges=False).connectivity()

dist_inlet, dist_outlet = [], []
edge_pt_ids = []
faces = ao_tagged.faces.reshape(-1, 4)[:, 1:]
for i in np.unique(edges['RegionId']):
    edge = edges.threshold([i, i+0.5], scalars='RegionId').extract_surface()
    edge_pt_ids.append(np.array(faces[ao_tagged.find_containing_cell(edge.points)]).flatten())

    dist_inlet.append(np.linalg.norm(np.array(edge.center) - np.array(cut_cyl_inlet.center)))
    dist_outlet.append(np.linalg.norm(np.array(edge.center) - np.array(cut_cyl_desc.center)))

id_closest_to_inlet = np.unique(edges['RegionId'])[np.argmin(dist_inlet)]
id_closest_to_desc = np.unique(edges['RegionId'])[np.argmin(dist_outlet)]
for i in np.unique(edges['RegionId']):
    edge = edges.threshold([i, i+0.5], scalars='RegionId').extract_surface()
    if i == id_closest_to_inlet: tag = 4
    elif i == id_closest_to_desc: tag = 2
    else: tag = 3
    ao_tagged['tags'][edge_pt_ids[i]] = tag

Creating subregion tags...


In [41]:
if show_plots:
    pl = pv.Plotter()
    pl.add_mesh(ao_tagged, scalars='tags', opacity=1, show_edges=True)
    pl.show()

In [42]:
##------------------------ Save data to file
io.save_to_file(ao_tagged, osp.join(output_dir, 'aorta.vtp'), clear_data=False)
io.save_to_file(centerlines, osp.join(output_dir, 'centerlines.vtp'), clear_data=False)
io.save_to_file(parentCenterline, osp.join(output_dir, 'parentCenterline.vtp'), clear_data=False)

print('Finished in {:.1f} seconds.'.format(time() - tic))

Finished in 174.6 seconds.
